In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [47]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk

In [48]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [76]:
# Import Dataset
df = pd.read_json('../data/newspaper/prothom_alo_2019-09-03.txt')
# print(df.content.unique())
df.head()

,content
0,\nরোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ: যথা...
1,\nরাবিতে ভর্তি পরীক্ষার আবেদন শুরু কাল থেকে\n ...
10,\nমাকে পুড়িয়ে মারার অভিযোগে মাদকাসক্ত ছেলের বি...
11,\nসন্ধ্যার মধ্যেই জামিনে মুক্তি পাচ্ছেন আয়শা\...
12,\nআরামে সহজে ফ্যাশনে\n -> \n\n\nফ্যাশন আর আরাম...


In [77]:
# Convert to list
data = df.content.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

[' রোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ: যথাযথ পদক্ষেপ নেবে অ্যামটব -> '
 'মুঠোফোন অপারেটরদের সংগঠন অ্যাসোসিয়েশন অব মোবাইল টেলিকম অপারেটরস অব বাংলাদেশ '
 '(অ্যামটব) বলেছে, জাতীয় পরিচয় তথ্যভান্ডারে (এনআইডি) সংরক্ষিত তথ্যের সঙ্গে '
 'আঙুলের ছাপ যাচাই করার পরই কেবল মোবাইলের সিম সক্রিয় হয়। আজ রোববার বাংলাদেশ '
 'টেলিযোগাযোগ নিয়ন্ত্রণ কমিশন (বিটিআরসি) মোবাইল অপারেটরদের চিঠি দিয়ে '
 'রোহিঙ্গাদের মুঠোফোন সেবা বন্ধের নির্দেশ দেওয়ার পর এক প্রতিক্রিয়ায় এ কথা '
 'জানায় অ্যামটব। অ্যামটবের মহাসচিব এস এম ফরহাদ বলেন, ‘মোবাইল অপারেটরেরা সব '
 'সময়ই বিটিআরসির নির্দেশনা মেনে চলে। এ ব্যাপারে তারা নিজেদের আয়ত্তের মধ্যে '
 'সম্ভব যথাযথ পদক্ষেপ নেবে।’ তিনি আরও বলেন, ‘আরেকটি বিষয় বলা দরকার, এনআইডি '
 'তথ্যভান্ডারের সঙ্গে আঙুলের ছাপ মিলিয়ে নিশ্চিত হওয়ার পরেই কেবল মোবাইল সিম '
 'সক্রিয় করা হয়।’ বিটিআরসির চিঠিতে বলা হয়েছে, রোহিঙ্গা ক্যাম্পে কোনো প্রকার '
 'সিম বিক্রি করা যাবে না। পাশাপাশি রোহিঙ্গাদের মুঠোফোন সুবিধা দেওয়ার বিষয়েও '
 'নিষেধ করেছে বিটিআরসি। এ বিষয়ে আগামী সাত কার্যদিবসের মধ্যে ব্যবস্থা নিয়ে '
 '

In [78]:
def valid_bengali_letters(char):
    return ord(char) >= 2433 and ord(char) <= 2543 

def get_replacement(char):
    if valid_bengali_letters(char):
        return char
    newlines = [10, 2404, 2405, 2551, 9576]
    if ord(char) in newlines: 
        return ' '
    return ' ';

def get_valid_lines(line):
    copy_line = ''
    for letter in line:
        copy_line += get_replacement(letter)
    return copy_line

In [79]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(nltk.word_tokenize(get_valid_lines(sentence)))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', 'যথাযথ', 'পদক্ষেপ', 'নেবে', 'অ্যামটব', 'মুঠোফোন', 'অপারেটরদের', 'সংগঠন', 'অ্যাসোসিয়েশন', 'অব', 'মোবাইল', 'টেলিকম', 'অপারেটরস', 'অব', 'বাংলাদেশ', 'অ্যামটব', 'বলেছে', 'জাতীয়', 'পরিচয়', 'তথ্যভান্ডারে', 'এনআইডি', 'সংরক্ষিত', 'তথ্যের', 'সঙ্গে', 'আঙুলের', 'ছাপ', 'যাচাই', 'করার', 'পরই', 'কেবল', 'মোবাইলের', 'সিম', 'সক্রিয়', 'হয়', 'আজ', 'রোববার', 'বাংলাদেশ', 'টেলিযোগাযোগ', 'নিয়ন্ত্রণ', 'কমিশন', 'বিটিআরসি', 'মোবাইল', 'অপারেটরদের', 'চিঠি', 'দিয়ে', 'রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', 'দেওয়ার', 'পর', 'এক', 'প্রতিক্রিয়ায়', 'এ', 'কথা', 'জানায়', 'অ্যামটব', 'অ্যামটবের', 'মহাসচিব', 'এস', 'এম', 'ফরহাদ', 'বলেন', 'মোবাইল', 'অপারেটরেরা', 'সব', 'সময়ই', 'বিটিআরসির', 'নির্দেশনা', 'মেনে', 'চলে', 'এ', 'ব্যাপারে', 'তারা', 'নিজেদের', 'আয়ত্তের', 'মধ্যে', 'সম্ভব', 'যথাযথ', 'পদক্ষেপ', 'নেবে', 'তিনি', 'আরও', 'বলেন', 'আরেকটি', 'বিষয়', 'বলা', 'দরকার', 'এনআইডি', 'তথ্যভান্ডারের', 'সঙ্গে', 'আঙুলের', 'ছাপ', 'মিলিয়ে', 'নিশ্চিত', 'হওয়ার', 'পরেই', 'কেব

In [80]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', 'যথাযথ', 'পদক্ষেপ', 'নেবে', 'অ্যামটব', 'মুঠোফোন', 'অপারেটরদের', 'সংগঠন', 'অ্যাসোসিয়েশন', 'অব', 'মোবাইল', 'টেলিকম', 'অপারেটরস', 'অব', 'বাংলাদেশ', 'অ্যামটব', 'বলেছে', 'জাতীয়', 'পরিচয়', 'তথ্যভান্ডারে', 'এনআইডি', 'সংরক্ষিত', 'তথ্যের', 'সঙ্গে', 'আঙুলের', 'ছাপ', 'যাচাই', 'করার', 'পরই', 'কেবল', 'মোবাইলের', 'সিম', 'সক্রিয়', 'হয়', 'আজ', 'রোববার', 'বাংলাদেশ', 'টেলিযোগাযোগ', 'নিয়ন্ত্রণ', 'কমিশন', 'বিটিআরসি', 'মোবাইল', 'অপারেটরদের', 'চিঠি', 'দিয়ে', 'রোহিঙ্গাদের', 'মুঠোফোন', 'সেবা', 'বন্ধের', 'নির্দেশ', 'দেওয়ার', 'পর', 'এক', 'প্রতিক্রিয়ায়', 'এ', 'কথা', 'জানায়', 'অ্যামটব', 'অ্যামটবের', 'মহাসচিব', 'এস', 'এম', 'ফরহাদ', 'বলেন', 'মোবাইল', 'অপারেটরেরা', 'সব', 'সময়ই', 'বিটিআরসির', 'নির্দেশনা', 'মেনে', 'চলে', 'এ', 'ব্যাপারে', 'তারা', 'নিজেদের', 'আয়ত্তের', 'মধ্যে', 'সম্ভব', 'যথাযথ', 'পদক্ষেপ', 'নেবে', 'তিনি', 'আরও', 'বলেন', 'আরেকটি', 'বিষয়', 'বলা', 'দরকার', 'এনআইডি', 'তথ্যভান্ডারের', 'সঙ্গে', 'আঙুলের', 'ছাপ', 'মিলিয়ে', 'নিশ্চিত', 'হওয়ার', 'পরেই', 'কেবল

In [81]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 2), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 2), (14, 1), (15, 1), (16, 3), (17, 1), (18, 2), (19, 2), (20, 1), (21, 1), (22, 4), (23, 1), (24, 1), (25, 2), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 2), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 2), (64, 1), (65, 1), (66, 3), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 2), (74, 1), (75, 2), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 3), (90, 5), (91, 1), (92, 2), (93, 2), (94, 1), (95, 1), (96, 3), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 3), (108, 1), (109, 1), (110, 1)

In [82]:
id2word[1]

'অপারেটরস'

In [83]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('অপারেটরদের', 2),
  ('অপারেটরস', 1),
  ('অপারেটরেরা', 1),
  ('অব', 2),
  ('অ্যামটব', 3),
  ('অ্যামটবের', 1),
  ('অ্যাসোসিয়েশন', 1),
  ('আইন', 1),
  ('আইনশৃঙ্খলা', 1),
  ('আগামী', 1),
  ('আগেও', 1),
  ('আঙুলের', 2),
  ('আজ', 1),
  ('আরও', 2),
  ('আরেকটি', 1),
  ('আয়ত্তের', 1),
  ('এ', 3),
  ('এই', 1),
  ('এক', 2),
  ('এনআইডি', 2),
  ('এম', 1),
  ('এস', 1),
  ('ও', 4),
  ('কথা', 1),
  ('কমিটি', 1),
  ('কমিশন', 2),
  ('করতে', 1),
  ('করা', 2),
  ('করার', 1),
  ('করেছে', 1),
  ('কার্যদিবসের', 1),
  ('কিন্তু', 1),
  ('কেবল', 2),
  ('কোনো', 1),
  ('ক্যাম্প', 1),
  ('ক্যাম্পে', 2),
  ('খবর', 1),
  ('গণমাধ্যমে', 1),
  ('গুরুত্ব', 1),
  ('গোয়েন্দা', 1),
  ('চলে', 1),
  ('চিঠি', 2),
  ('চিঠিতে', 2),
  ('ছাপ', 2),
  ('জন', 1),
  ('জব্বারের', 1),
  ('জাতীয়', 1),
  ('জানাতে', 1),
  ('জানানো', 1),
  ('জানায়', 1),
  ('টেলিকম', 1),
  ('টেলিযোগাযোগ', 2),
  ('ডাক', 1),
  ('তথ্য', 1),
  ('তথ্যভান্ডারে', 1),
  ('তথ্যভান্ডারের', 1),
  ('তথ্যের', 1),
  ('তারা', 1),
  ('তিনি', 1),
  ('দরকার', 1),
  ('দিয়ে

In [84]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [85]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"কোম্পানিটির" + 0.015*"ও" + 0.015*"অলিম্পিক" + 0.014*"বিস্কুট" + '
  '0.011*"থেকে" + 0.011*"রয়েছে" + 0.009*"হাজার" + 0.009*"ব্যাটারির" + '
  '0.009*"কনফেকশনারি" + 0.008*"এ"'),
 (1,
  '0.001*"ও" + 0.001*"থেকে" + 0.001*"হয়েছে" + 0.001*"সময়" + 0.001*"করে" + '
  '0.001*"করা" + 0.001*"সেকেন্ড" + 0.001*"করেন" + 0.001*"বাংলাদেশ" + '
  '0.001*"পোশাক"'),
 (2,
  '0.027*"সময়" + 0.027*"সেকেন্ড" + 0.016*"১০" + 0.016*"নিয়ে" + 0.012*"১০০" + '
  '0.010*"বাংলাদেশের" + 0.010*"৯" + 0.009*"সালে" + 0.009*"বিশ্ব" + '
  '0.009*"হাজার"'),
 (3,
  '0.023*"জিপিএ" + 0.019*"থেকে" + 0.019*"আবেদন" + 0.015*"হবে" + '
  '0.015*"প্রাথমিক" + 0.012*"৩" + 0.012*"ভর্তি" + 0.012*"সহ" + 0.012*"মোট" + '
  '0.012*"ন্যূনতম"'),
 (4,
  '0.001*"ও" + 0.001*"হয়" + 0.001*"সঙ্গে" + 0.001*"বলা" + 0.001*"হয়েছে" + '
  '0.001*"এ" + 0.001*"মধ্যে" + 0.001*"না" + 0.001*"থেকে" + 0.000*"সিম"'),
 (5,
  '0.018*"রিয়েলিটি" + 0.016*"শো" + 0.014*"টেলিভিশন" + 0.011*"করে" + '
  '0.011*"দেশের" + 0.010*"অনুষ্ঠান" + 0.010*"দর্শকের" + 0.009*

In [86]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.487470229483947

Coherence Score:  0.49482675727544334
